In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 0. Configure Package Dependencies

In [ ]:
import shutil
import random
import gc
import multiprocessing
import copy
import time
import numpy as np
import pandas as pd
from zipfile import ZipFile
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

## 1. Import the Dataset 

In [ ]:
def unzip(fileName, dirName):
    """
    Extract all files from a zip file to a certain Directory.
    """
    with ZipFile(fileName, 'r') as zipObj:
        zipObj.extractall(dirName)   

In [ ]:
# Define training and test zip file path.
TRAIN_ZIP = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip'
TEST_ZIP = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip'

# Define training and test directory.
DIR = '/kaggle/working/'

unzip(TRAIN_ZIP, DIR)
unzip(TEST_ZIP, DIR)

In [ ]:
# Define training and validation directories.
TRAIN_DIR = '/kaggle/working/train/'
TEST_DIR = '/kaggle/working/test/'

train_dogs = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]
test_imgs = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

# Generate training set by sampling a slice of dogs and cats. 
# train_imgs = train_dogs[:2500] + train_cats[:2500] # get 10%
train_imgs = train_dogs + train_cats
random.shuffle(train_imgs)  

del train_dogs
del train_cats
gc.collect()

In [ ]:
# Check file name and file path.
print('Total {} train images'.format(len(train_imgs)))
print()
print(os.listdir(TRAIN_DIR)[:5])
print()
print(train_imgs[:5])

In [ ]:
# Check file name and file path.
print('Total {} test images'.format(len(test_imgs)))
print()
print(os.listdir(TEST_DIR)[:5])
print()
print(test_imgs[:5])

## 2. Preview the Dataset 

In [ ]:
idx = 0
nrows = 3
ncols = 3
figsize = (16,16)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
for i in range(nrows):
    for j in range(ncols):
        axs[i,j].imshow(Image.open(train_imgs[idx]))
        idx += 1
        axs[i,j].axes.get_xaxis().set_visible(False)
        axs[i,j].axes.get_yaxis().set_visible(False)        
plt.show()        

## 3. Cross-Validation

In [ ]:
train_set, valid_set = train_test_split(train_imgs, test_size=0.2, random_state=42)

del train_imgs
gc.collect()

In [ ]:
# Print train list and validation list.
print('Number of training set: {}'.format(len(train_set)))
print('Number of validation set: {}'.format(len(valid_set)))

## 4. Load Datasets

### 4.1 DataLoader

In [ ]:
class dataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_path = self.data[idx]
        img = Image.open(img_path)
        if self.transform:
            img_aug = self.transform(img)
            
        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label = 1
        elif label == 'cat':
            label = 0
        
        return img_aug, label      
        

### 4.2. Image Augumentation

In [ ]:
input_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val':
    transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [ ]:
print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets.
image_datasets = {
    'train': dataset(train_set, transform=data_transforms['train']),
    'val': dataset(valid_set, transform=data_transforms['val'])
}

batch_size = 32
# Switch to perform multi-process data loading
num_workers = multiprocessing.cpu_count()
print('num_workers = {}'.format(num_workers))

# Create training and validation dataloaders.
dataloaders_dict = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=num_workers)
}

In [ ]:
print('{} train data in total'.format(len(image_datasets['train'])))
print('{} iters per epoch for train'.format(len(dataloaders_dict['train'])))
print()
print('{} val data in total'.format(len(image_datasets['val'])))
print('{} iters per epoch for val'.format(len(dataloaders_dict['val'])))
print()
print('Image shape is {}'.format(image_datasets['train'][0][0].shape))

## 5. Build Model

### 5.1 Transfer Model

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Number of epochs to train for 
num_epochs = 20

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Set model parameters, requires_grad attribute
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False 

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    if model_name == "resnet":
        model_ft = models.resnet18(pretrained=use_pretrained)  # download pretrained model and parameters
        set_parameter_requires_grad(model_ft, feature_extract) # don't update gradient form pretrained model
        num_ftrs = model_ft.fc.in_features                     # get last layer dimension
        model_ft.fc = nn.Linear(num_ftrs, num_classes)         # create new dense layer
        input_size = 224                                # resnet18's input should be 224
    return model_ft, input_size

In [ ]:
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
print(model_ft)

In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr = 1e-4, eps = 1e-6)
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [ ]:
# def adjust_learning_rate(optimizer, epoch):
#     """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
#     lr = learning_rate * (0.1**(epoch // 30))
#     for param_group in optimizer.state_dict()['param_groups']:
#         param_group['lr'] = lr

## 6. Define Train Model

In [ ]:
def train_model(model, dataloaders, optimizer, criterion, num_epochs=5):
    since = time.time()
    history = {
        'train_acc': [],
        'train_loss': [],
        'val_acc': [],
        'val_loss': [] 
    }
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs-1))
        print("-" * 10)
        
        for phase in ["train", "val"]:
            running_loss = 0.0
            running_acc = 0.0
            if phase == "train":
                model.train()
            else:
                model.eval()
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                with torch.autograd.set_grad_enabled(phase=="train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0) # 
                #running_acc += torch.sum(preds.view(-1) == labels.view(-1).item())
                running_acc += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_acc / len(dataloaders[phase].dataset)
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
            if phase == 'val':
                history['val_acc'].append(epoch_acc)
                history['val_loss'].append(epoch_loss)
            if phase == 'train':
                history['train_acc'].append(epoch_acc)
                history['train_loss'].append(epoch_loss)
        
        print()
    
    time_elapsed = time.time() - since
    print("Training compete in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
    print("Best val Acc: {:4f}".format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

## 7. Run Model

### 7.1 Run Training and Validation Step

In [ ]:
# Train and evaluate
model_ft, ohist = train_model(model_ft, dataloaders_dict, optimizer, criterion, num_epochs=num_epochs)

### 7.2 Visualize Training Results

In [ ]:
train_acc = ohist['train_acc']
val_acc = ohist['val_acc']
train_loss = ohist['train_loss']
val_loss = ohist['val_loss']
epochs_range = range(num_epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### 8.3 Save Model

In [ ]:
save_model = True
if (save_model):
    save_path = os.path.join(DIR, 'model_save')
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    torch.save(model_ft.state_dict(), save_path + '/resnet18.pth') 

## 9. Prediction

In [ ]:
test_transform = data_transforms['val']

id_list = []
pred_list = []

with torch.no_grad():
    for test_path in tqdm(test_imgs):
        img = Image.open(test_path)
        img_id = int(test_path.split('/')[-1].split('.')[0])
        img = test_transform(img)
        img = img.unsqueeze(0)
        img = img.to(device)
        
        model_ft.eval()
        outputs = model_ft(img)
        preds = F.softmax(outputs, dim=1)[:, 1].tolist() # predict vale
        
        id_list.append(img_id)
        pred_list.append(preds[0])
        

## 10. Submission

In [ ]:
my_submission = pd.DataFrame({'id': id_list, 'label': pred_list})

my_submission.sort_values(by='id', inplace=True)
my_submission.reset_index(drop=True, inplace=True)

my_submission.to_csv('submission_pytorch_resnet18.csv', index=False)